In [29]:
# =========================================================
# SEÇÃO 1: IMPORTAÇÕES E SETUP GERAL
# =========================================================

In [30]:
import os
import numpy as np
import pandas as pd
import warnings
from tqdm import tqdm
from functools import partial

# --- Nossas funções auxiliares agora são importadas! ---

from helper.utils import (
    definir_seed,
    carregar_serie,
    dividir_serie_temporal,
    preparar_dados_para_neuralforecast
)
# --- Importações específicas para o treinamento ---

from statsmodels.tsa.arima.model import ARIMA
import pmdarima as pm
from neuralforecast import NeuralForecast
from neuralforecast.models import NBEATS, MLP, LSTM, NHITS, GRU
warnings.filterwarnings("ignore")

In [31]:
# =========================================================
# SEÇÃO 2: PIPELINE DE EXPERIMENTO
# =========================================================

In [32]:
def encontrar_melhor_arima_auto(treino_log, freq):
    # (Mantenha sua função original aqui)
    print("Buscando melhor ordem ARIMA com auto_arima...")
    m = 12 if freq.startswith('M') else (4 if freq.startswith('Q') else 1)
    auto_arima_model = pm.auto_arima(treino_log, m=m, seasonal=True, trace=False,
                                     error_action='ignore', suppress_warnings=True, stepwise=True)
    print(
        f"Melhor ordem encontrada: {auto_arima_model.order} Sazonal: {auto_arima_model.seasonal_order}")
    return auto_arima_model.order, auto_arima_model.seasonal_order

In [ ]:
def executar_experimento(nome_da_serie, horizonte):
    try:
        # --- 1. CONFIGURAÇÕES INICIAIS ---
        print("="*80)
        print(
            f"INICIANDO EXECUÇÃO PARA: Dataset='{nome_da_serie}', Horizonte={horizonte}")
        print("="*80)

        SEED = 42
        definir_seed(SEED)
        INPUT_SIZE_NEURAL = 50
        MAX_STEPS_NEURAL = 200

        # --- 2. CARGA E PREPARAÇÃO DOS DADOS ---
        serie_original = carregar_serie(nome_da_serie)
        percentual_treino = 1 - (horizonte / len(serie_original))
        treino_orig, teste_orig = dividir_serie_temporal(
            serie_original, percentual_treino=percentual_treino)

        if len(treino_orig) < INPUT_SIZE_NEURAL + horizonte:
            print(
                f"AVISO: A série '{nome_da_serie}' é muito curta para os parâmetros. Pulando.")
            return None

        serie_log = np.log(serie_original)
        treino_log, _ = dividir_serie_temporal(
            serie_log, percentual_treino=percentual_treino)
        freq = serie_original.index.freqstr or pd.infer_freq(
            serie_original.index)
        if freq is None:
            return None
        previsoes_teste = {'y_true': teste_orig.values}

        # --- 3. MODELO ARIMA ---
        modelo_arima = None
        try:
            print(f"Processando: ARIMA")
            ordem, ordem_sazonal = encontrar_melhor_arima_auto(
                treino_log, freq)
            modelo_arima = ARIMA(treino_log.asfreq(
                freq), order=ordem, seasonal_order=ordem_sazonal).fit()
            previsoes_teste['ARIMA'] = np.exp(
                modelo_arima.forecast(steps=horizonte)).values
        except Exception as e:
            print(f"AVISO CRÍTICO: ARIMA falhou. Abortando esta execução.")
            return None

        # --- 4. MODELOS NEURAIS PUROS ---
        print("\n--- Processando Modelos Neurais Puros ---")
        df_treino_log_nf = preparar_dados_para_neuralforecast(
            treino_log, nome_da_serie)
        modelos_puros_para_testar = {
            'N-BEATS': NBEATS, 'MLP': MLP, 'LSTM': LSTM, 'GRU': GRU, 'NHITS': NHITS}
        for nome_modelo, classe_modelo in modelos_puros_para_testar.items():
            try:
                print(f"Processando: {nome_modelo} (Puro)")
                modelo_neural = [classe_modelo(input_size=INPUT_SIZE_NEURAL, h=horizonte,
                                               max_steps=MAX_STEPS_NEURAL, scaler_type='standard', random_seed=SEED)]
                nf = NeuralForecast(models=modelo_neural, freq=freq)
                nf.fit(df=df_treino_log_nf, verbose=False)
                previsoes_teste[nome_modelo] = np.exp(
                    nf.predict()[classe_modelo.__name__].values)
            except Exception as e:
                previsoes_teste[nome_modelo] = np.nan

        # --- 5. PREPARAÇÃO DOS RESÍDUOS E EXECUÇÃO DOS HÍBRIDOS ---
        print("\n--- Processando Modelos Híbridos ---")
        residuos_treino_log = modelo_arima.resid
        df_residuos_treino = preparar_dados_para_neuralforecast(
            residuos_treino_log, "residuos")

        modelos_hibridos_mimo = {
            'Híbrido N-BEATS (MIMO)': NBEATS,
            'Híbrido MLP (MIMO)': MLP,
            'Híbrido LSTM (MIMO)': LSTM,
            'Híbrido GRU (MIMO)': GRU  # <-- Novo modelo adicionado
        }

        for nome_modelo, classe_modelo in modelos_hibridos_mimo.items():
            try:
                print(f"Processando: {nome_modelo}")
                # Apenas executa se os dados forem suficientes para a estratégia MIMO
                if len(df_residuos_treino) >= INPUT_SIZE_NEURAL + horizonte:
                    modelo_hibrido = [classe_modelo(
                        input_size=INPUT_SIZE_NEURAL, h=horizonte, max_steps=MAX_STEPS_NEURAL, scaler_type='standard', random_seed=SEED)]
                    nf = NeuralForecast(models=modelo_hibrido, freq=freq)
                    nf.fit(df=df_residuos_treino, verbose=False)
                    preds_residuos = nf.predict(
                    )[classe_modelo.__name__].values
                    previsoes_teste[nome_modelo] = previsoes_teste['ARIMA'] + \
                        preds_residuos
                else:
                    previsoes_teste[nome_modelo] = np.nan
            except Exception as e:
                print(f"AVISO: {nome_modelo} falhou: {e}")
                previsoes_teste[nome_modelo] = np.nan

        # (Mantivemos a estratégia Recursiva com N-HITS como uma alternativa estável ao N-BEATS)
        try:
            print("Processando: Híbrido N-HITS (Recursive)")
            modelo_rec = [NHITS(input_size=INPUT_SIZE_NEURAL, h=1,
                                max_steps=MAX_STEPS_NEURAL, scaler_type='standard', random_seed=SEED)]
            nf_rec = NeuralForecast(models=modelo_rec, freq=freq)
            nf_rec.fit(df=df_residuos_treino, verbose=False)
            preds_df = nf_rec.predict(h=horizonte)
            previsoes_teste['Híbrido N-HITS (Recursive)'] = previsoes_teste['ARIMA'] + \
                preds_df['NHITS'].values
        except Exception as e:
            previsoes_teste['Híbrido N-HITS (Recursive)'] = np.nan

        # --- 6. FINALIZAÇÃO ---
        df_final = pd.DataFrame(previsoes_teste, index=teste_orig.index)
        df_final['dataset'] = nome_da_serie
        df_final['horizonte'] = horizonte
        return df_final.reset_index().rename(columns={'index': 'ds'})

    except Exception as e:
        import traceback
        print(f"ERRO GERAL: {e}")
        traceback.print_exc()
        return None

In [34]:
# =========================================================
# SEÇÃO 3: ORQUESTRADOR (LÓGICA DA CAMADA SILVER)
# =========================================================

In [35]:
# --- Configurações ---
# LISTA_DE_DATASETS = ['AirPassengers', 'co2', 'nottem', 'JohnsonJohnson', 'UKgas', 'Sunspots','Nile','ukdriverdeaths']
LISTA_DE_DATASETS = ['AirPassengers', 'co2']
VETOR_DE_HORIZONTES = [10, 12]
output_dir = "./data/silver"
os.makedirs(output_dir, exist_ok=True)
output_file = os.path.join(output_dir, "resultados_completos.csv")

# --- Lógica de Execução Incremental ---
if os.path.exists(output_file):
    os.remove(output_file)

header_escrito = False

In [ ]:
for dataset in tqdm(LISTA_DE_DATASETS, desc="Processando Datasets"):
    for horizonte in tqdm(VETOR_DE_HORIZONTES, desc=f"Testando Horizontes para {dataset}", leave=False):
        # Executa o experimento para uma combinação de dataset e horizonte
        df_resultado_detalhado = executar_experimento(dataset, horizonte)

        if df_resultado_detalhado is not None:
            # Lógica para salvar incrementalmente no arquivo da camada Silver
            if not header_escrito:
                # Escreve o cabeçalho apenas na primeira vez
                df_resultado_detalhado.to_csv(
                    output_file, index=False, mode='w', header=True)
                header_escrito = True
            else:
                # Anexa os novos resultados sem o cabeçalho
                df_resultado_detalhado.to_csv(
                    output_file, index=False, mode='a', header=False)

print(f"\nProcesso finalizado. Resultados salvos em '{output_file}'")

Processando Datasets:   0%|          | 0/2 [00:00<?, ?it/s]


INICIANDO EXECUÇÃO PARA: Dataset='AirPassengers', Horizonte=10
Cache não encontrado. Buscando dados de 'AirPassengers' via statsmodels...
-> Salvando cópia do dataset 'AirPassengers' em cache: ./data/bronze\airpassengers.csv
Processando: ARIMA
Buscando melhor ordem ARIMA com auto_arima...
Melhor ordem encontrada: (2, 0, 0) Sazonal: (0, 1, 1, 12)


Seed set to 42
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name         | Type          | Params | Mode 
-------------------------------------------------------
0 | loss         | MAE           | 0      | train
1 | padder_train | ConstantPad1d | 0      | train
2 | scaler       | TemporalNorm  | 0      | train
3 | blocks       | ModuleList    | 2.5 M  | train
-------------------------------------------------------
2.5 M     Trainable params
1.3 K     Non-trainable params
2.5 M     Total params
9.983     Total estimated model params size (MB)
31        Modules in train mode
0         Modules in eval mode



--- Processando Modelos Neurais Puros ---
Processando: N-BEATS (Puro)
Epoch 199: 100%|██████████| 1/1 [00:00<00:00,  3.89it/s, v_num=0, train_loss_step=0.00844, train_loss_epoch=0.00844]

`Trainer.fit` stopped: `max_steps=200` reached.


...

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name         | Type          | Params | Mode 
-------------------------------------------------------
0 | loss         | MAE           | 0      | train
1 | padder_train | ConstantPad1d | 0      | train
2 | scaler       | TemporalNorm  | 0      | train
3 | blocks       | ModuleList    | 2.5 M  | train
-------------------------------------------------------
2.5 M     Trainable params
0         Non-trainable params
2.5 M     Total params
9.981     Total estimated model params size (MB)
34        Modules in train mode
0         Modules in eval mode


Epoch 199: 100%|██████████| 1/1 [00:00<00:00,  2.62it/s, v_num=56, train_loss_step=0.0152, train_loss_epoch=0.0152]

`Trainer.fit` stopped: `max_steps=200` reached.


Epoch 199: 100%|██████████| 1/1 [00:00<00:00,  2.56it/s, v_num=56, train_loss_step=0.0152, train_loss_epoch=0.0152]



INICIANDO EXECUÇÃO PARA: Dataset='co2', Horizonte=12
Processando: ARIMA
Buscando melhor ordem ARIMA com auto_arima...
Melhor ordem encontrada: (3, 1, 2) Sazonal: (2, 0, 2, 12)
